<a id='top'></a>

# CSCI 3022: Intro to Data Science - Spring 2020 Practicum 2
***

This practicum is due on Canvas by **11:59 PM on Friday May 1**. Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  

**Here are the rules:** 

1. All work, code and analysis, must be your own. 
1. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
1. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
1. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions though.
2. You may **NOT** post to message boards or other online resources asking for help.
3. You may **NOT** copy-paste solutions *from anywhere*.
4. You may **NOT** collaborate with classmates or anyone else.
5. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

***

**Name**:  JC Abrahamson

***


**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.


---

In [95]:
from scipy import stats
from math import isnan
import numpy as np 
import statsmodels.api as sm
import pandas as pd
import statistics
import matplotlib.pyplot as plt
%matplotlib inline
from calendar import month_name, different_locale

<br>

---

### [50 points] Problem 1: Multiple Linear Regression to Explain House Hauntings

<img src="https://s-media-cache-ak0.pinimg.com/originals/09/72/01/09720128cff5de4d4af038cd3fcf7f69.jpg" style="width: 300px;"/>

In an effort to control the skyrocketing prices of real estate in the Colorado Front Range, Governor Polis implemented a cutting edge new intervention. This new program oversaw the introduction of ghosts back into their natural ecosystem, after the ghost population seriously dwindled in recent decades due to overhaunting. However, an unfortunate miscalculation has led to haunted houses becoming a very serious problem in Colorado. Modern problems require modern solutions, so Governor Polis has hired you and the famous hedgehog data scientist/part-time ghostbuster Amy to determine what features of a house may be used to best predict a `haunted` score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted).

You decide to use multiple linear regression to understand and predict what factors lead to increased haunted house hazard. You collected a data set from Haunted Zillow, the lesser-known database of haunted house prices and attributes. The data cover a variety of potential features, and you'll find this data in the file `houses.csv`. 

**Response**: 

- $\texttt{haunted}$: a haunting score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted)

**Features**: 

- $\texttt{age}$: age of the house, in years
- $\texttt{area}$: square footage of interior of house
- $\texttt{bathrooms}$: number of bathrooms
- $\texttt{distance metro}$: distance to the nearest major metropolitan area (in miles)
- $\texttt{distance cemetery}$: distance to the nearest cemetery (in miles)
- $\texttt{cats}$: the number of cats within a one-block radius of the house
- $\texttt{howls}$: the number of wolf howls heard on an average night in the house's neighborhood
- $\texttt{clouds}$: what percentage of the sky was covered by clouds (fraction, 0-1)
- $\texttt{precipitation}$: amount of precipitation in the past 72 hours (inches)
- $\texttt{misery index}$: an economic indicator for how miserable the average United States citizen is, based on the unemployment rate and the inflation rate. More [here](https://www.stuffyoushouldknow.com/podcasts/whats-the-misery-index.htm) and [here](https://en.wikipedia.org/wiki/Misery_index_(economics)). Higher values correspond to more miserable citizens.
- $\texttt{ice cream sold}$: the number of units of ice cream sold at the farmer's market the week the house was most recently sold

**Part A**: Read the data from `houses.csv` into a Pandas DataFrame.  Note that since we will be doing a multiple linear regression we will need all of the features, so you should drop any row in the DataFrame that is missing data. 

In [189]:
df_house = pd.read_csv("houses.csv")
df_house = df_house.dropna()
df_house = df_house.reset_index(drop=True)
df_house.head(len(df_house))

,age,area,bathrooms,distance metro,distance cemetery,cats,howls,clouds,precipitation,misery index,ice cream sold,haunted
0,118.94,2113.0,4,7.1,3.23,7.0,3.0,1.00,0.82,12.99,273.0,0.123131
1,20.60,1773.0,2,7.4,4.19,5.0,5.0,1.00,0.99,16.77,184.0,-0.200243
2,260.84,2511.0,3,7.0,3.38,2.0,0.0,1.00,1.17,16.49,141.0,0.258651
3,101.33,2000.0,2,7.9,4.05,6.0,8.0,0.13,0.92,8.28,146.0,0.147983
4,96.94,1476.0,1,7.5,5.75,4.0,1.0,1.00,1.73,5.90,178.0,-0.113756
...,...,...,...,...,...,...,...,...,...,...,...,...
67,237.66,1403.0,2,6.6,5.97,8.0,2.0,1.00,2.16,6.85,271.0,0.352572
68,65.39,1880.0,4,4.1,3.59,7.0,2.0,0.60,1.67,16.83,211.0,0.331913
69,56.24,1996.0,2,7.4,3.38,4.0,4.0,1.00,0.71,6.42,215.0,0.627473
70,4.87,1732.0,1,7.1,3.96,7.0,2.0,1.00,1.46,19.44,122.0,0.411308


**Part B**: Perform the appropriate statistical test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  Clearly describe your methodology and show all computations in Python. 

$\large$$ H_0: \beta_1 = \beta_2 = \ ... \ = \beta_n = 0$

$\large$$ H_1: \beta_k \neq 0 \ \text{for at least one value of k in 1, 2, ... , p}$

$\large$$ SSE = \sum^n_{i=1} (y_i - \hat y)^2$

$\large$$ SST = \sum^n_{i=1} (y_i - \bar y)^2$

$\large$$ F = \frac{\frac{SST - SSE}{p}}{\frac{SSE}{n - p - 1}} = 16.2$

$\large$$ df_{\text{model}} = 11$

$\large$$ df_{\text{residuals}} = 60$

$\large$$ \text{p-value} = 1-stats.f.cdf(F, df_{\text{model}}, df_{\text{residuals)}} = 0.00000000000003752553823$

$\large$$ \text{p-value} < 0.01$
                                       
We **reject** the null hypothesis.

In [174]:
x = df_house[['age', 'area', 'bathrooms', 'distance metro', 'distance cemetery', 'cats', 'howls', 'clouds', 'precipitation', 'misery index', 'ice cream sold']]
x = sm.add_constant(x)
y = df_house['haunted']
mod = sm.OLS(y, x).fit()
df_model = mod.df_model
df_resid = mod.df_resid
F = mod.fvalue
pval = 1-stats.f.cdf(F, df_model, df_resid)
print("df_model:", df_model)
print("df_residuals:", df_resid)
print("F:", F)
print("P-value: {:.23f}".format(pval))

#https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html

df_model: 11.0
df_residuals: 60.0
F: 16.224860847419265
P-value: 0.00000000000003752553823


**Part C**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=5$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may of course use canned routines like statmodels OLS, but you may not call any Python method that explicitly performs forward selection.

In [183]:
def SLR(var, df, resp_str):
    SLR = []
    for v in range(len(var)):
        bhat, ahat, rval, pval, stderr = stats.linregress(df[var[v]], df[resp_str])
        y = ahat + bhat * df[var[v]]
        SLR.append(y)
    return SLR

def forward_select(df, resp_str, maxk):
    var = list(df.columns)
    var.remove(resp_str)
    y_i = df[resp_str]
    bestSSE = []
    for v in range(len(var)):
        y = SLR(var, df, resp_str)[v]
        currSSE = 0
        currSSE = sum((y_i - y)**2)
        bestSSE.append(currSSE)
    stage = 1
    m = bestSSE.index(min(bestSSE))
    B0 = var[m]
    features = []
    features.append(B0)
    print("Added feature '{}' at stage {}.".format(B0, stage))
    var.remove(B0)
    stage += 1
    maxk -= 1
    for i in range(maxk):
        sse = []
        names = []
        for j in var:
            if j not in features:
                x = df[[j] + features]
                x = sm.add_constant(x)
                m = sm.OLS(y_i, x).fit()
                s = sum((y_i - m.predict(x))**2)
                sse.append(s)
                names.append(j)
        m = sse.index(min(sse))
        feature_name = names[m]
        features.append(feature_name)
        print("Added: '{}' - Stage {}.".format(feature_name, stage))  
        stage += 1     
    return features

selected_features = forward_select(dfHouses, 'haunted', 5)

Added feature 'distance cemetery' at stage 1.
Added: 'cats' - Stage 2.
Added: 'age' - Stage 3.
Added: 'misery index' - Stage 4.
Added: 'distance metro' - Stage 5.


**Part D**: Write down the multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

In [184]:
x = dfHouses[selected_features]
x = sm.add_constant(x)
y = dfHouses['haunted']
mod = sm.OLS(y, x).fit()
vals = mod.params

print("ŷ = {} + ( {} * distance cemetery ) + ( {} * age ) + () {} * cats ) + () {} * bathrooms ) + ( {} * misery_index )".format(vals[0], vals[1], vals[2], vals[3], vals[4], vals[5]))

ŷ = 0.3848475810219145 + ( -0.11144808095644886 * distance cemetery ) + ( 0.034762361151591425 * age ) + () 0.0012470094773047516 * cats ) + () 0.005756304970505458 * bathrooms ) + ( -0.02690422199419283 * misery_index )


**Part E**: Perform the appropriate statistical test at the $\alpha = 0.05$ significance level to determine whether there is a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in **Part D**. 

$\large$$ H_0: \beta_{\text{area}} = \beta_{\text{distance metro}} = \beta_{\text{howls}} = \beta_{\text{clouds}} = \beta_{\text{percipitation}} = \beta_{\text{ice cream sold}} = 0$

$\large$$ H_1: \beta_k \neq 0 \ \text{for at least one k in} \ \{\text{area, distance metro, howls, clouds, percipitation, ice cream sold}\}$

$\large$$ F = \dfrac{\frac{(SSE_{reduced} - SSE_{full})}{(p-k)}}{\frac{SSE_{full}}{(n-p-1)}} = 0.223288926$

$\large$$ p-value = 1 - stats.f.cdf(F, p-k, n-p-1) = 0.9677403$

$\large$$ p-value > 0.05$


So we **fail to reject** the null hypothesis and determine that there is not a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in Part D. So, the reduced model is better.

In [180]:
y = dfHouses["haunted"]
reduced = selected_features
xreduced = dfHouses[reduced]
xreduced = sm.add_constant(xreduced)
n = len(dfHouses)
p = len(dfHouses.columns)-1
k = len(selected_features)
reduced_model = sm.OLS(y, xreduced).fit()
data = list(dfHouses.columns)
data.remove('haunted')
xdata = dfHouses[data]
xdata = sm.add_constant(xdata)
data_model = sm.OLS(y, xdata).fit()
SSE_reduced = sum((y - reduced_model.predict(xreduced))**2)
SSE_data = sum((y - data_model.predict(xdata))**2)
F_num = (SSE_reduced - SSE_data)/(p-k)
F_denom = (SSE_data)/(n-p-1)
F = F_num/F_denom
pval = 1-stats.f.cdf(F, p-k, n-p-1)

print("F:", F)
print("P-value:", pval)

#https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html
#https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

F: 0.22328892612193924
P-value: 0.9677403220482735


**Part F**: Based on your conclusions in **Part E**, use the _better_ of the two models to predict the haunted house hazard when the following features are observed: 

- $\texttt{age}$: 150 years
- $\texttt{area}$: 2200 square feet
- $\texttt{bathrooms}$: 3 bathrooms
- $\texttt{distance metro}$: 25 miles
- $\texttt{distance cemetery}$: 5 miles
- $\texttt{cats}$: 20 cats
- $\texttt{howls}$: 5 wolf howls/night
- $\texttt{clouds}$: 0.65 cloud cover
- $\texttt{precipitation}$: 0 inches
- $\texttt{misery index}$: 10
- $\texttt{ice cream sold}$: 125

reduced model is better

In [186]:
allX = {
    'age': 150,
    "𝚊𝚛𝚎𝚊" : 2200, 
    "bathrooms" : 3, 
    "distance metro" : 25, 
    "distance cemetery" : 5,
    "cats" : 20,
    "𝚑𝚘𝚠𝚕𝚜" : 5,
    "𝚌𝚕𝚘𝚞𝚍𝚜" : 0.65, 
    "𝚙𝚛𝚎𝚌𝚒𝚙𝚒𝚝𝚊𝚝𝚒𝚘𝚗" : 0, 
    "misery index" : 10,
    "𝚒𝚌𝚎 𝚌𝚛𝚎𝚊𝚖 𝚜𝚘𝚕𝚍" : 125
}

y_hat = betas[0]
rest_betas = betas.drop('const')
for i in range(len(selected_features)):
    y_hat += allX[selected_features[i]] * rest_betas[i]
    
print("ŷ = {}".format(y_hat))

ŷ = 0.0948633207174453


**Part G:** Governor Polis dabbles a bit in the art of data science, as well as the science of data art. He tells you that the response (`haunted` score) that you and Amy predicted is actually the natural logarithm of the _odds_ that a house with the given features is haunted, where if $p$ is the probability that a house is haunted, then the odds are given by $$\text{odds} = \dfrac{p}{1-p}$$

So using Govenor Polis's information and combining it with our knowledge of logistic regression, the probability of a house being haunted is:

$$p(\text{haunted} \mid x_1, x_2, \ldots) = \dfrac{1}{1+\texttt{exp}[-(\beta_0 + \beta_1x_1 + \ldots + \beta_p x_p)]}$$

Perform this computation, then use a decision threshold of 0.5 to classify the house from **Part F** as haunted or not haunted. No new models should be fit here; use the same model that you used in Part F.

$\large$$ \text{odds} = \dfrac{p}{1-p} = e^{\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5}$

We are concerned with probability of being haunted or not. This can be done by fitting a Log Regression Model to our $\hat{y}$.

$\large$$ p(y = 1 | x) = sigma(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_p x_p)$


**Decision Threshold = 0.5:**

$\large$$ p(y = 1 | x) = sigma(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5)$

**Predict:**

$\hat{y} = 
\begin{cases}
        1 = \text{haunted} &amp; \text{if} \ sigma(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5) \geq 0.5 \\
        0 = \text{not haunted} &amp; \text{if} \ sigma(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5) < 0.5 
\end{cases}$

**Features:**

$\large$$ x_1 = \text{distance cemetery}$

$\large$$ x_2 = \text{age}$

$\large$$ x_3 = \text{cats}$

$\large$$ x_4 = \text{bathrooms}$

$\large$$ x_5 = \text{misery index}$

**Classify if House is haunted (Part F):**

$\large \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5)}} = 0.5237$


$\large 0.5237 > 0.5$

The house from Part F **is** haunted.

In [182]:
exp = np.exp(-y_hat)
denom = 1 + exp
num = 1
print("{} > 0.5".format(num/denom))

0.5236980611752295 > 0.5


<br>

---



### [50 points] Problem 2: Amazon Forest Fires
A non-profit trying to protect the amazon rain forest has recruited you to join their data science corps. For your first task, they've given you a dataset with the number of reported forest fires in each state in the Amazon region of Brazil during each month between 1998 and 2017. The Brazilian government has 500 extra wildland firefighters and they have asked your non-profit to determine which state or states they should allocate these firefighters to during each month of the year. To do this, they want you to calculate an 80% confidence interval for the mean and median number of fires that occur during each month for each state, and use those statistics to determine where the firefighters should be assigned.

#### Part A: Loading The CSV
Read the csv located in `amazon.csv` into a pandas data frame. Brazil and many other countries use the period (.) symbol as a thousands separator and a comma (,) as the decimal separator. Ex. One Thousand And $\frac{75}{100}$ would be represented as $1.000,75$ instead of the familiar english notation $1,000.75$. When you read it in, you'll need to use a period(.) as the thousands separator and a comma(,) as the decimal separator. Because the comma is already in use as the decimal separator, this file uses a different character to separate columns in the data. Open up the file in a text editor and figure out what character was used. Then find the correct arguments to `pd.read_csv` to read in this file properly. Look up the docs if you're unsure what the arguments you'll need are. Print out the `.info` summary of the dataframe after you've read it in.

In [129]:
amazon_fires = pd.read_csv('amazon.csv', sep=";", thousands='.', decimal ='/', engine='python')
amazon_fires.info()
amazon_fires.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
year      6453 non-null object
state     6452 non-null object
month     6454 non-null object
number    6448 non-null object
date      6450 non-null object
dtypes: object(5)
memory usage: 252.2+ KB


,year,state,month,number,date
0,1998,Acre,Janeiro,0,1998-01-01
1,1999,Acre,Janeiro,0,1999-01-01
2,2000,Acre,Janeiro,0,2000-01-01
3,2001,Acre,Janeiro,0,2001-01-01
4,2002,Acre,Janeiro,0,2002-01-01
5,2003,Acre,Janeiro,10,2003-01-01
6,2004,Acre,Janeiro,0,2004-01-01
7,2005,Acre,Janeiro,12,2005-01-01
8,2006,Acre,Janeiro,4,2006-01-01
9,2007,Acre,Janeiro,0,2007-01-01


#### Part B: Data Cleaning

This dataset isn't paticularly useful in it's current state, so we'll need to clean it up a bit. Some data scientists say that most of their job is to wrangle data, so this will give you a taste of cleaning a real world data set. Perform the following tasks. 
1. Drop the 'date' column. The only information this column holds is the year, which we already have in another column. Use the `.info` summary provided to check your work.
2. Drop any rows with null values in any of the remaining columns. Use the provided code to print the number of rows remaining after this step.
3. Print all the unique values of the 'month' column. You'll notice that one is encoded with a different character encoding then the format that pandas is using.
4. Convert the Portugese month names to English month names. If you'd like to use them, we've included the 'month_name' and the 'different_encoding' modules of the python calendar library. There are many ways to accomplish this task, and these modules are not required, but may make things easier. As part of this step, you should make sure that the Portugese month with the encoding problem is translated to the correct english month. Use the `.unique` method provided for you to check your work. 
5. Check the number column for any values that seem impossible. If you find any values you think are impossible, drop them. As a guidline, we would never expect a single state to have more than 50,000 reported forest fires in a single month. Also keep in mind that we are tracking forest fires here. Do negative or fractional forest fires really make sense? You should check for any obivously impossible conditions that you think might occur, and drop rows accordingly. Use the provided code to print the number of rows remaining after this step.
6. Since you're new on the job, some of your co-workers may have played a prank on you... Print out all the unique values of the 'year' column and drop any rows with values that don't make sense. Use the provided code to print the number of rows remaining after this step.
7. For every state in the data, print the number of rows the state has associated with it. A number of states have far more observations than the others. Each state should have roughly 240 observations (20 years multiplied by 12 months/year minus any bad data). Drop all the observations for any states that have more than 240 rows associated with them. For two points of extra credit, figure out why these states have way more rows associated with them than they should. If you choose to do the extra credit, put your answer in the markdown cell below. 
8. To give you an idea of whether your answer is correct, we've provided a unit test below the last cell. It should pass. If it doesn't, go back and figure out which step has gone awry.

We've given you a code cell for each task to make organizing the grading a bit easier. Please perform step 1 in the first code cell and so on.

**NOTE:** Since some of these tasks are not totally trivial, you may use any resources other than your classmates on this part of this problem. This means you may consult google, stack overflow, the python/pandas documentation, some random book on pandas you might have, etc... But you may not consult your classmates for help. We will also be more helpful on this problem in office hours and in response to your *private* piazza messages.  ***CITE ALL RESOURCES USED IN A CODE COMMENT. A URL OR A BOOK TITLE IS SUFFICIENT. ANY CODE OBIVOUSLY COPIED FROM OUTSIDE SOURCES WITH OUT A CITATION WILL EARN YOU NO CREDIT ON THIS PROBLEM.***

In [130]:
#Code for data cleaning task 1 here:
amazon_fires = amazon_fires.drop(['date'], axis=1) 
amazon_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 4 columns):
year      6453 non-null object
state     6452 non-null object
month     6454 non-null object
number    6448 non-null object
dtypes: object(4)
memory usage: 201.8+ KB


In [131]:
#Code for data cleaning task 2 here:
amazon_fires = amazon_fires.dropna()
print(len(amazon_fires))

6446


In [132]:
#Code for data cleaning task 3 here:
months = amazon_fires.month.unique()
print(months)

['Janeiro' 'Fevereiro' 'Marï¿½o' 'Abril' 'Maio' 'Junho' 'Julho' 'Agosto'
 'Setembro' 'Outubro' 'Novembro' 'Dezembro']


In [133]:
#Code for data cleaning task 4 here:
amazon_fires['month'] = np.where(amazon_fires['month'] == "Janeiro", "January", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Fevereiro", "February", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Marï¿½o", "March", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Abril", "April", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Maio", "May", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Junho", "June", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Julho", "July", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Agosto", "August", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Setembro", "September", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Outubro", "October", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Novembro", "November", amazon_fires['month'])
amazon_fires['month'] = np.where(amazon_fires['month'] == "Dezembro", "December", amazon_fires['month'])
print("Row Count: {}".format(amazon_fires.count()))
#https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html

Row Count: year      6446
state     6446
month     6446
number    6446
dtype: int64


In [188]:
#Code for data cleaning task 5
amazon_fires = amazon_fires[~amazon_fires.number.str.contains(',')]
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html
amazon_fires["number"] = pd.to_numeric(amazon_fires["number"])
ind = amazon_fires[ amazon_fires['number'] > 50000].index
amazon_fires.drop(ind , inplace=True)
ind2 = amazon_fires[ amazon_fires['number'] < 0].index
amazon_fires.drop(ind2 , inplace=True)

print(len(amazon_fires))

AttributeError: Can only use .str accessor with string values!

In [187]:
#Code for data cleaning task 6
year = amazon_fires.year.unique()
print(year)

ind4 = amazon_fires[ amazon_fires['year'] == '1000bc'].index
amazon_fires.drop(ind4, inplace=True)

ind5 = amazon_fires[ amazon_fires['year'] == '-40'].index
amazon_fires.drop(ind5, inplace=True)

ind6 = amazon_fires[ amazon_fires['year'] == '10bc'].index
amazon_fires.drop(ind6, inplace=True)

#dropna for the nan?
amazon_fires.dropna()
# 
ind8 = amazon_fires[ amazon_fires['year'] == "our new data scientist won't notice this"].index
amazon_fires.drop(ind8, inplace=True)
print(len(amazon_fires))

['1998' '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007'
 '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017']
4772


In [136]:
#Code for data cleaning task 7
amazon_fires = amazon_fires.groupby('state').filter(lambda x : len(x)<=240)
#https://stackoverflow.com/questions/38544301/filter-data-with-groupby-in-pandas
print(len(amazon_fires))

4772


In [137]:
#Given Test. This Cell Should Not Throw an Exception!
assert \
    len(amazon_fires['state'].unique()) == 20 and \
    list(amazon_fires['month'].unique()) == \
        ['January', 'February', 'March', 'April', 'May', 'June',
             'July', 'August','September', 'October', 'November',
             'December'] and \
    len(amazon_fires) == 4772, 'somethings wrong in problem 1.'

#### Part C: Medians and Means!
In this part of the problem, we'll calculate an 80% confidence interval for both the mean and median number of wildfires each state has during each month of the year. 

For the mean you should use the appropriate confidence interval with the correct distribution. Remember to check how many observations we have. Use the sample standard deviation. 

For the median, we'll have to bootstrap it because the median is not known to be normally distributed. You should bootstrap 1000 samples of the same length as the original sample for each month for each state. Calculate the median for each bootstrapped sample. Then take the middle 80% of the bootstrapped medians as your confidnce interval. This is called a bootstrapped percentile median. There are a few more complex and slightly more rigourous ways to estimate the median from bootstrapped samples, but this will serve for our purposes.

You're given a dictionary of dictionaries to store your confidence intervals for the medians and means in. 

Take a look at the dictionary structure below. 

The idea here is that for every month, for every state, you will fill in the `mean_CI` with a length two list that contains the low and high end of the confidence interval for the true mean number of fires for that state in that month. 

Similiarly, for every month, for every state, you will fill in the `median_CI` with a length two list that contains the low and high end of the confidence interval for the true median number of fires for that state in that month.

For example:

When you're done `months['January']['Acre']['mean_CI']` should be a list with the low and high bounds for the confidence interval of the true mean number of wildfires in the state of Acre in January. So `months['January']['Acre']['mean_CI'][0]` should be the low end of the CI for the mean, and `months['January']['Acre']['mean_CI'][1]` should be the high end of the CI for the mean.

`months['January']['Acre']['median_CI']` should hold the confidence interval for the true median number of wildfires in the state of Acre in January. So `months['January']['Acre']['median_CI'][0]` should be the low end of the CI for the median, and `months['January']['Acre']['median_CI'][1]` should be the high end of the CI for the median.

In [145]:
#GIVEN CODE DO NOT CHANGE THIS!!!
#YOU SHOULD BE WRITING CODE IN THE NEXT CELL(s) THAT FILLS IN THE 'months' DICTIONARY.

#If you're curious what copy and deep copy do and why we used them here see an explanation 
#here: https://thispointer.com/python-how-to-copy-a-dictionary-shallow-copy-vs-deep-copy/

from copy import deepcopy

mean_median_dict ={
    'mean_CI' : None,
    'median_CI': None
}

CI_median_num_fires = {
    'Acre': dict(mean_median_dict),
    'Alagoas':dict( mean_median_dict),
    'Amapa':dict( mean_median_dict),
    'Amazonas':dict( mean_median_dict),
    'Bahia':dict( mean_median_dict),
    'Ceara':dict( mean_median_dict),
    'Distrito Federal':dict( mean_median_dict),
    'Espirito Santo':dict( mean_median_dict),
    'Goias':dict( mean_median_dict),
    'Maranhao':dict( mean_median_dict),
    'Minas Gerais':dict( mean_median_dict),
    'Para':dict( mean_median_dict),
    'Pernambuco':dict( mean_median_dict),
    'Piau':dict( mean_median_dict),
    'Rondonia':dict( mean_median_dict),
    'Roraima':dict( mean_median_dict),
    'Santa Catarina':dict( mean_median_dict),
    'Sao Paulo':dict( mean_median_dict),
    'Sergipe':dict( mean_median_dict),
    'Tocantins':dict( mean_median_dict)  
}

months = {
    'January': deepcopy(CI_median_num_fires),
    'February': deepcopy(CI_median_num_fires),
    'March': deepcopy(CI_median_num_fires), 
    'April': deepcopy(CI_median_num_fires), 
    'May': deepcopy(CI_median_num_fires),
    'June': deepcopy(CI_median_num_fires),
    'July': deepcopy(CI_median_num_fires),
    'August': deepcopy(CI_median_num_fires), 
    'September': deepcopy(CI_median_num_fires), 
    'October': deepcopy(CI_median_num_fires),
    'November': deepcopy(CI_median_num_fires),
    'December': deepcopy(CI_median_num_fires)
}

In [152]:
#Your code here:

import scipy.stats


def mean_confidence_interval(data, confidence=0.80):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
#https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data

state_list = amazon_fires.state.unique()
month_list = amazon_fires.month.unique()

tmp=[]
for month in month_list:
    for state in state_list:
        medians = []
        for x in range(0,1000):
            tmp = CI_median_num_fires[state]
            resample = np.random.choice(tmp,50,replace = True)
            resample_median = np.median(resample)
            medians.append(resample_median)
        interval = [np.percentile(medians,5),np.percentile(medians,95)]
        CI_median_num_fires[state]= interval
    months[month] = CI_median_num_fires


    #Got Stuck



ValueError: 'a' must be 1-dimensional or an integer

In [149]:
#DONT CHANGE THIS. WE USE IT TO MAKE THE OUTPUT LEGIBLE FOR GRADING
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(months)

{'April': {'Acre': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                    'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Alagoas': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                       'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Amapa': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                     'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Amazonas': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                        'median_CI': [348.6416263808994, 1432.3583736191006]},
           'April': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                     'median_CI': [348.6416263808994, 1432.3583736191006]},
           'August': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                      'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Bahia': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                     'm

           'November': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                        'median_CI': [348.6416263808994, 1432.3583736191006]},
           'October': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                       'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Para': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                    'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Pernambuco': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                          'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Piau': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                    'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Rondonia': {'mean_CI': [432.8727756532602, 1548.3732769783187],
                        'median_CI': [348.6416263808994, 1432.3583736191006]},
           'Roraima': {'mean_CI': [432.8727756532602, 1548.3732769783187],
         

In [36]:
#Given Test for the mean confidence intervals
rounded_mean_CI = [round(x, 2) for x in months['April']['Acre']['mean_CI']]
assert rounded_mean_CI == [0.76, 3.34], 'somethings wrong in the mean'

AssertionError: somethings wrong in the mean

In [ ]:
#Given test for the median confidence intervals. 
#Your code is probably correct if it passes this test, but since bootstrapping the medain is a stochastic process
#you may have this test fail. If it fails, run it a few times. 
#If it continues to fail, your code is probably incorrect.

low_median_CI = months['April']['Acre']['median_CI'][0]
high_median_CI = months['April']['Acre']['median_CI'][1]
assert -1 <= low_median_CI <= 1 and 0 <= high_median_CI <= 3, 'somethings wrong in the median'

#### Part E: Where Do The Firefighters Go?
Now, we'll determine which state the Brazilian government should assign it's fire fighters to. For each month of the year, you should perform the folllowing selection process:
1. Find the state with the highest CI for the median for this month (it's easiest and ok to just use the upper bound here). 
2. Find any states that have a median CI that overlaps with the highest CI foud in step 1. If no states overlap with the highest CI found in step 1, then use that stat. 
3. If overlapping confidence intervals are found on the median, we'll use the CI for the mean to break ties.
4. Out of the states with overlapping CIs for median (every state in part 3), find the state with the highest mean CI. 
5. Determine if any of the states from part 3 have a mean CI that overlaps with the state found in step 4. 
6. If no state overlap with the state found in part 4, then just use that state. If other states have overlapping mean CIs too, then we'll split up the firefighters and assign some of them to every state that has both an overlapping median and mean CI with the state that has the highest median CI.

Once you've used the selection process above, use a markdown table to display a list of each state that recieves  some of the firefighters for each month.

In [ ]:
# Your code here.